In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.datasets import fetch_california_housing
import model
from model import read_data_model, add_laplace_noise

np.random.seed(42)

# Cargar los datos
all_data = fetch_california_housing()
X_train_val, X_val, X_test, Y_train_val, Y_val, Y_test = read_data_model(all_data, 'MedInc')

# Escalado de los datos
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Hiperparámetros
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
mse1 = []

# Selección del mejor alpha mediante validación cruzada
for i in alpha:
    model_lasso = Lasso(alpha=i, max_iter=10000)
    model_lasso.fit(X_train_val_scaled, Y_train_val)
    Y_predict = model_lasso.predict(X_val_scaled)
    mse1.append(mse(Y_val, Y_predict))

# Selección del mejor alpha
best_alpha = alpha[np.argmin(mse1)]
print(f"Best alpha: {best_alpha}")

# Entrenamiento del modelo Lasso con el mejor alpha sin ruido
model_original = Lasso(alpha=best_alpha, max_iter=10000)
model_original.fit(X_train_val_scaled, Y_train_val)
Y_predict = model_original.predict(X_test_scaled)

# Evaluar el modelo original (sin ruido)
MSE_original = mse(Y_test, Y_predict)
MAE_original = mae(Y_test, Y_predict)
RMSE_original = np.sqrt(MSE_original)
R2_original = r2(Y_test, Y_predict)

print("\nPredicción de precios de casas sin ruido:")
print(f"MSE: {MSE_original}")
print(f"MAE: {MAE_original}")
print(f"RMSE: {RMSE_original}")
print(f"R2: {R2_original}")

# Paso 2: Crear nuevos datos de entrada reemplazando solo la columna de ingreso con las predicciones
column_index = list(all_data.feature_names).index('MedInc')  # Índice de la columna de ingreso
X_train_val_new = np.copy(X_train_val)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento para el modelo original
Y_predict_train = model_original.predict(X_train_val_scaled)

# Reemplazar solo la columna de ingreso
Y_train_val_new = X_train_val_new[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new[:, column_index] = Y_predict_train  
X_test_new[:, column_index] = Y_predict 

scaler_reconstruct = StandardScaler()
X_train_val_new_scaled = scaler_reconstruct.fit_transform(X_train_val_new)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Reconstrucción de la columna de ingreso usando los datos sin ruido
model_reconstruct = Lasso(alpha=best_alpha, max_iter=10000)
model_reconstruct.fit(X_train_val_new_scaled, Y_train_val_new)
income_predict = model_reconstruct.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (sin ruido)
MSE_income_reconstruct = mse(X_test[:, column_index], income_predict)
MAE_income_reconstruct = mae(X_test[:, column_index], income_predict)
RMSE_income_reconstruct = np.sqrt(MSE_income_reconstruct)
R2_income_reconstruct = r2(X_test[:, column_index], income_predict)

print("\nReconstrucción de ingresos (sin ruido):")
print(f"MSE: {MSE_income_reconstruct}")
print(f"MAE: {MAE_income_reconstruct}")
print(f"RMSE: {RMSE_income_reconstruct}")
print(f"R2: {R2_income_reconstruct}")

# Paso 4: Añadir ruido Laplaciano a la columna de ingreso y repetir el procedimiento
epsilon = 10  # Usamos el epsilon ya seleccionado
X_train_val_noisy = add_laplace_noise(X_train_val, epsilon)
scalerRuido = StandardScaler()
X_train_val_noisy_scaled = scalerRuido.fit_transform(X_train_val_noisy)
X_test_scaled = scalerRuido.transform(X_test)

# Entrenar el modelo con datos ruidosos
model_noisy = Lasso(alpha=best_alpha, max_iter=10000)
model_noisy.fit(X_train_val_noisy_scaled, Y_train_val)
Y_predict_noisy = model_noisy.predict(X_test_scaled)

# Evaluar el modelo con datos ruidosos
MSE_noisy = mse(Y_test, Y_predict_noisy)
MAE_noisy = mae(Y_test, Y_predict_noisy)
RMSE_noisy = np.sqrt(MSE_noisy)
R2_noisy = r2(Y_test, Y_predict_noisy)

print("\nPredicción de precios con ruido:")
print(f"MSE: {MSE_noisy}")
print(f"MAE: {MAE_noisy}")
print(f"RMSE: {RMSE_noisy}")
print(f"R2: {R2_noisy}")

# Paso 5: Crear nuevos datos de entrada reemplazando la columna de ingreso con las predicciones ruidosas
X_train_val_new_noisy = np.copy(X_train_val_noisy)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento con el modelo ruidoso
Y_predict_train_noisy = model_noisy.predict(X_train_val_noisy_scaled)

# Reemplazar solo la columna de ingreso con las predicciones ruidosas
Y_train_val_new = X_train_val_new_noisy[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new_noisy[:, column_index] = Y_predict_train_noisy  
X_test_new[:, column_index] = Y_predict

scaler_reconstruct = StandardScaler()
X_train_val_new_scaled_noisy = scaler_reconstruct.fit_transform(X_train_val_new_noisy)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Reconstrucción de la columna de ingreso con datos ruidosos
model_reconstruct_noisy = Lasso(alpha=best_alpha, max_iter=10000)
model_reconstruct_noisy.fit(X_train_val_new_scaled_noisy, Y_train_val_new)
income_predict_noisy = model_reconstruct_noisy.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (con ruido)
MSE_income_reconstruct_noisy = mse(X_test[:, column_index], income_predict_noisy)
MAE_income_reconstruct_noisy = mae(X_test[:, column_index], income_predict_noisy)
RMSE_income_reconstruct_noisy = np.sqrt(MSE_income_reconstruct_noisy)
R2_income_reconstruct_noisy = r2(X_test[:, column_index], income_predict_noisy)

print("\nReconstrucción de ingresos (con ruido):")
print(f"MSE: {MSE_income_reconstruct_noisy}")
print(f"MAE: {MAE_income_reconstruct_noisy}")
print(f"RMSE: {RMSE_income_reconstruct_noisy}")
print(f"R2: {R2_income_reconstruct_noisy}")


Best alpha: 0.001

Predicción de precios de casas sin ruido:
MSE: 0.3376051969378297
MAE: 0.4332968333001364
RMSE: 0.5810380339855814
R2: 0.6447444689662328

Reconstrucción de ingresos (sin ruido):
MSE: 0.00011488068598886334
MAE: 0.00812893817101378
RMSE: 0.010718240806627893
R2: 0.999951166113359

Predicción de precios con ruido:
MSE: 0.42636340635405146
MAE: 0.49359046223424036
RMSE: 0.6529650881586637
R2: 0.5513458924461778

Reconstrucción de ingresos (con ruido):
MSE: 6.119244783606927
MAE: 1.9155802743477897
RMSE: 2.4737107315947284
R2: -1.6011901262496413
